In [1]:
# Standard imports
import pandas as pd
import numpy as np
import time

# For multiindex slicing
idx = pd.IndexSlice

In [2]:
# Load counts df
ct_df = pd.read_csv('csv_results/oh_counts_table.csv', index_col=[0,1], usecols=range(1,9))
ct_df = ct_df.reorder_levels([1,0])

# Get rid of NpN entries
ix = ct_df.index.drop('NpN', level=1) 
ct_df = ct_df.loc[ix,:]

# Regularize counts
ct_df = ct_df.where(ct_df>0,0.0)+1
ct_df['total'] = ct_df.sum(axis=1)
ct_df.head()

,,4,5,6,7,8,9,total
replicate,primer,,,,,,,
rep1,ApA,1.153971,271.469849,3207.209324,2703.402852,1097.586942,556.130115,7836.953055
rep2,ApA,55.453680,246.168443,4052.189959,4813.025962,3522.058243,646.429137,13335.325424
rep3,ApA,41.758572,205.025771,3232.493927,2683.401060,1356.395298,28.347419,7547.422046
rep1,ApC,18.881950,159.577889,3050.709273,5012.971234,130.377103,1.000000,8373.517450
rep2,ApC,38.747765,300.890478,3749.775250,7071.634930,2677.158825,197.929395,14036.136644


In [3]:
# Compute percentages for 
pct_df = ct_df.copy()
pct_df.loc[:,:] = 0

# Compute percent for each replicate
for rep in ['rep1', 'rep2', 'rep3']:
    rep_sum = ct_df.loc[idx[rep,:],:].sum(axis=0)
    N = rep_sum.sum()
    pct_df.loc[idx[rep,:],:] = 100*ct_df.loc[idx[rep,:],:]/N
pct_df['total'] = pct_df.sum(axis=1)

# Save
file_name = 'csv_results/oh_pct_by_rep.csv'
pct_df.to_csv(file_name)
print(f'Wrote {file_name}')

# Show
pct_df.loc[idx['rep2',:],:]

Wrote csv_results/oh_pct_by_rep.csv


4         5         6         7         8         9  \
replicate primer                                                               
rep2      ApA     0.005944  0.026387  0.434361  0.515916  0.377535  0.069292   
          ApC     0.004153  0.032253  0.401944  0.758020  0.286969  0.021216   
          ApG     0.001582  0.022282  0.696035  1.149085  0.635407  0.165566   
          ApU     0.003422  0.030519  0.255056  0.330341  0.282238  0.090531   
          CpA     0.000807  0.028474  0.396422  0.467051  0.153149  0.000107   
          CpC     0.000886  0.020373  0.525909  0.755182  0.114552  0.000107   
          CpG     0.000837  0.027037  0.350925  0.457366  0.077637  0.009568   
          CpU     0.003502  0.030714  0.509172  0.508497  0.145262  0.006628   
          GpA     0.005612  0.025486  1.164661  1.340979  0.785299  0.142648   
          GpC     0.002172  0.008267  0.856043  1.377772  0.720098  0.035305   
          GpG     0.002171  0.051431  3.423051  3.513820  0.980300  0.064387   
          GpU     0.006896  0.042189  0.830142  1.302982  0.800412  0.076793   
          UpA     0.023155  0.428362  5.006924  5.025430  1.786231  0.141389   
          UpC     0.005998  0.069145  1.031712  2.127261  0.315031  0.006116   
          UpG     0.016059  0.405406  0.867733  0.993682  0.210165  0.025768   
          UpU     0.045269  0.199935  1.003689  1.252590  0.244293  0.023521   

                      total  
replicate primer             
rep2      ApA      2.858869  
          ApC      3.009112  
          ApG      5.339915  
          ApU      1.984214  
          CpA      2.092021  
          CpC      2.834018  
          CpG      1.846742  
          CpU      2.407551  
          GpA      6.929373  
          GpC      5.999314  
          GpG     16.070318  
          GpU      6.118824  
          UpA     24.822983  
          UpC      7.110526  
          UpG      5.037628  
          UpU      5.538593

In [4]:
# Compute and save mean counts
mean_pct_df = pct_df.groupby(level=1).mean()

# Save
file_name = 'csv_results/oh_pct_mean.csv'
mean_pct_df.to_csv(file_name)
print(f'Wrote {file_name}')

# Show
mean_pct_df

Wrote csv_results/oh_pct_mean.csv


,4,5,6,7,8,9,total
primer,,,,,,,
ApA,0.003884,0.028645,0.415321,0.398172,0.229551,0.046346,2.243837
ApC,0.002281,0.026573,0.387288,0.647415,0.133066,0.007156,2.407560
ApG,0.002715,0.023282,0.685212,1.035211,0.603771,0.149790,4.999961
ApU,0.003881,0.024835,0.232888,0.209582,0.145140,0.067785,1.368222
CpA,0.001293,0.028121,0.411853,0.486201,0.139861,0.000120,2.134898
CpC,0.001422,0.021094,0.539921,0.779290,0.122883,0.000120,2.929460
CpG,0.001028,0.030974,0.363309,0.467835,0.079011,0.008081,1.900476
CpU,0.003710,0.032454,0.513388,0.483922,0.127227,0.002461,2.326323
GpA,0.004935,0.023797,1.190992,1.243117,0.636192,0.100547,6.399163


In [5]:
# Compute and save mean counts
std_pct_df = pct_df.groupby(level=1).std()

# Save
file_name = 'csv_results/oh_pct_std.csv'
std_pct_df.to_csv(file_name)
print(f'Wrote {file_name}')

# Show
std_pct_df

Wrote csv_results/oh_pct_std.csv


,4,5,6,7,8,9,total
primer,,,,,,,
ApA,0.003250,0.003116,0.030261,0.103658,0.130644,0.036900,0.538191
ApC,0.001852,0.006876,0.022104,0.095819,0.139345,0.012177,0.534672
ApG,0.000987,0.001982,0.026970,0.111219,0.042747,0.017740,0.324797
ApU,0.002527,0.006027,0.022722,0.105104,0.122561,0.036129,0.539148
CpA,0.000424,0.001604,0.034442,0.034043,0.012172,0.000013,0.123966
CpC,0.000826,0.001975,0.033725,0.045590,0.021503,0.000013,0.136480
CpG,0.000243,0.003728,0.027301,0.026456,0.008754,0.001689,0.090595
CpU,0.001088,0.001634,0.036015,0.023624,0.015633,0.003618,0.119371
GpA,0.001025,0.005025,0.070455,0.136177,0.132756,0.038335,0.485350
